# Learning trajectories and initial state

Code to reproduce the Figures 2, 3, 5 and 6.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from cycler import cycler
from matplotlib.axes import Axes
from matplotlib.figure import Figure
from matplotlib.ticker import StrMethodFormatter
import matplotlib.transforms as trns
from pathlib import Path

from plearning.phone_pair import PhonePair
from plearning.utils import native_nonnative_selection, make_pairwise_score, query

plt.style.use("./paper.mplstyle")  # Comment if you don't have LaTeX installed

MODE, COLORS, MARKERS, LINESTYLES = "within", ["#0072B2", "#E69F00"], ["o", "d"], ["solid", "dashed"]

root = Path("./results/abx")

initial_state_noise = pd.read_csv(root / "initial_state_noise.csv")
initial_state_untrained = pd.read_csv(root / "initial_state_untrained.csv")
initial_state_noise = initial_state_noise[query(initial_state_noise, phone_pair=None, mode=MODE)]
initial_state_untrained = initial_state_untrained[query(initial_state_untrained, phone_pair=None, mode=MODE)]

no_pretraining =    pd.read_csv(root / "no_pretraining.csv")
noise_pretrained = pd.read_csv(root / "noise_pretraining.csv")
crossling = pd.read_csv(root / "crossling_pretraining.csv")
scores = no_pretraining.merge(
    crossling,
    on=["test", "train", "phone_pair", "split", "idx", "mode"],
    suffixes=["", "_crossling"],
    how="outer",
).merge(
    noise_pretrained,
    on=["test", "train", "phone_pair", "split", "idx", "mode"],
    suffixes=["_baseline", "_noise"],
    how="outer",
)

wj, rl = PhonePair("[W]-[Y]"), PhonePair("[L]-[R]", reverse_print=True)
native_non_native = {
    None: native_nonnative_selection(scores, ["csj", "gpj", "buc", "wsj"], phone_pair=None, mode=MODE),
    rl: native_nonnative_selection(scores, ["buc", "wsj"], phone_pair=rl, mode=MODE),
    wj: native_nonnative_selection(scores, ["buc", "wsj"], phone_pair=wj, mode=MODE),
}

In [ ]:
def plot_trajectory(
    fig: Figure,
    ax: Axes,
    native_df: pd.DataFrame,
    non_native_df: pd.DataFrame,
    styles: dict,
) -> tuple[Figure, Axes]:
    for use_native, col in styles:
        df = native_df if use_native else non_native_df
        pairwise_scores = make_pairwise_score(df.groupby(["train", "idx", "split"], as_index=False)[col].mean(), col)
        mean = pairwise_scores.groupby("split")[col].mean()
        std = pairwise_scores.groupby("split")[col].std(ddof=0)
        style = styles[(use_native, col)]
        duration = 500 // mean.index
        ax.fill_between(duration, (1 - mean - std) * 100, (1 - mean + std) * 100, color=style["color"], alpha=0.1)
        ax.plot(duration, (1 - mean) * 100, label="Native" if use_native else "Non-native", **style)
    ax.set_xscale("log")
    ax.yaxis.set_major_formatter(StrMethodFormatter("{x:,.0f}"))
    ax.xaxis.set_major_formatter(StrMethodFormatter("{x}"))
    ax.set_xticks([1, 4, 20, 100, 500])
    ax.tick_params(axis="both")
    ax.minorticks_off()
    return fig, ax


def plot_initial_state(
    fig: Figure,
    ax: Axes,
    initial_state_untrained: pd.DataFrame,
    initial_state_noise: pd.DataFrame,
    styles: dict,
    linewidth: float = 1.0,
    width: float = 0.8,
    capsize: int = 5,
    dx: float = 0.3,
    dy: float = 0.1,
):
    mean = [(1 - initial_state_untrained.score.mean()) * 100, (1 - initial_state_noise.score.mean()) * 100]
    std = [initial_state_untrained.score.std(ddof=0) * 100, initial_state_noise.score.std(ddof=0) * 100]

    ax.set_ylim(45, 100)
    ax.bar(
        [0, 1],
        mean,
        yerr=std,
        color=COLORS,
        edgecolor="black",
        capsize=capsize,
        width=width,
        linewidth=linewidth,
        tick_label=list(styles.keys()),
        error_kw={"elinewidth": linewidth, "capthick": linewidth},
        zorder=2,
    )
    labels = ax.get_xticklabels()
    labels[0].set_fontsize("medium")
    labels[0].set_color(COLORS[0])
    labels[1].set_fontsize("medium")
    labels[1].set_color(COLORS[1])

    x, y = [0, 1], [
        (1 - initial_state_untrained.score.to_numpy()) * 100,
        (1 - initial_state_noise.score.to_numpy()) * 100,
    ]
    for i, style in enumerate(styles.values()):
        ax.scatter(
            x[i] + np.linspace(-width / 2 + 0.1, width / 2 - 0.1, len(y[i])),
            y[i],
            color=style["color"],
            edgecolor="black",
            linewidth=0.5,
            zorder=3,
            marker=style["marker"],
        )
    ax.yaxis.set_major_formatter(StrMethodFormatter("{x:,.0f}"))
    ax.set_ylabel("ABX accuracy (in \\%)")
    ax.set_title("Initial state")
    ax.grid(visible=False, axis="x")

    first, sec = ax.xaxis.get_majorticklabels()
    offset = trns.ScaledTranslation(-dx, -dy, fig.dpi_scale_trans)
    first.set_transform(first.get_transform() + offset)
    offset = trns.ScaledTranslation(dx, -dy, fig.dpi_scale_trans)
    sec.set_transform(sec.get_transform() + offset)

    ax.hlines(50, ax.get_xlim()[0], ax.get_xlim()[1], color="black", linestyles="dotted")
    ax.text(1.7, 50, r"\noindent Chance \\ level", fontsize="x-small", verticalalignment="center")
    return fig, ax

In [ ]:
legend_titles = ["No pretraining", r"\noindent Pretrained on \\ ambient sounds"]
initial_styles = dict(zip(legend_titles, cycler("color", COLORS) * cycler("marker", ["X"])))
labels = [(True, "score_baseline"), (False, "score_baseline"), (True, "score_noise"), (False, "score_noise")]
styles = dict(zip(labels, cycler("color", COLORS) * (cycler("marker", MARKERS) + cycler("ls", LINESTYLES))))

fig, ax = plt.subplots(
    figsize=(8, 3),
    nrows=1,
    ncols=2,
    gridspec_kw={"width_ratios": [1, 3], "wspace": 0.35, "left": 0.1, "right": 0.9, "top": 1, "bottom": 0.2},
)

plot_initial_state(fig, ax[0], initial_state_untrained, initial_state_noise, initial_styles)

native, non_native = native_non_native[None]
plot_trajectory(fig, ax[1], native, non_native, styles)
ax[1].set_ylabel("ABX accuracy (in \\%)")
ax[1].set_xlabel("Training hours")
ax[1].set_title("Development", fontsize="large")

handles, _ = ax[1].get_legend_handles_labels()
first_legend = fig.legend(
    title=legend_titles[0], handles=[handles[0], handles[1]], loc="lower left", bbox_to_anchor=(0.9, 0.6)
)
first_legend._legend_box.align = "left"
first_legend.get_title().set_color(COLORS[0])
second_legend = fig.legend(
    title=legend_titles[1], handles=[handles[2], handles[3]], loc="lower left", bbox_to_anchor=(0.9, 0.25)
)
second_legend.get_title().set_color(COLORS[1])
second_legend._legend_box.align = "left"

fig.text(0, 1.03, r"\textbf{a)}", fontsize="x-large", transform=fig.transFigure)
fig.text(0.35, 1.03, r"\textbf{b)}", fontsize="x-large", transform=fig.transFigure)

plt.show()

In [ ]:
legend_titles = ["No pretraining", r"\noindent Pretrained on \\ ambient sounds"]
labels = [(True, "score_baseline"), (False, "score_baseline"), (True, "score_noise"), (False, "score_noise")]
styles = dict(zip(labels, cycler("color", COLORS) * (cycler("marker", MARKERS) + cycler("ls", LINESTYLES))))

fig, ax = plt.subplots(
    figsize=(6, 4),
    ncols=2,
    nrows=2,
    sharex="all",
    sharey="col",
    gridspec_kw={"wspace": 0.15, "left": 0.15, "right": 1, "top": 0.75, "bottom": 0.15},
)

for i, phone_pair in enumerate([rl, wj]):
    native, non_native = native_non_native[phone_pair]
    for j, col in enumerate(["score_baseline", "score_noise"]):
        plot_trajectory(None, ax[i, j], native, non_native, {k: v for k, v in styles.items() if k[1] == col})

handles, labels = ax[0, 0].get_legend_handles_labels()
handles = [plt.plot([], marker="", ls="")[0]] + handles
labels = ["Training language: ", "American English", "Japanese"]
legend = fig.legend(
    handles=handles,
    labels=labels,
    loc="lower left",
    bbox_to_anchor=(0.1, 0.85),
    ncols=3,
    frameon=True,
    handlelength=3,
)
legend.legend_handles[1].set_color("black")
legend.legend_handles[2].set_color("black")

for vpack in legend._legend_handle_box.get_children()[:1]:
    for hpack in vpack.get_children():
        hpack.get_children()[0].set_width(0)

ax[0, 0].set_yticks([70, 75, 80, 85, 90, 95])
ax[0, 1].set_yticks([90, 92, 94, 96])
ax[0, 0].set_title(legend_titles[0], color=COLORS[0], fontsize="medium", pad=10)
ax[0, 1].set_title("Pretrained on ambient sounds", color=COLORS[1], fontsize="medium", pad=10)
ax[0, 1].text(1.1, 0.5, rl.tipa, transform=ax[0, 1].transAxes, fontsize="x-large")
ax[1, 1].text(1.1, 0.5, wj.tipa, transform=ax[1, 1].transAxes, fontsize="x-large")

fig.supylabel("ABX accuracy (in \\%)", x=0.05, y=0.45)
fig.supxlabel("Training hours", x=0.57, y=0)

plt.show()

In [ ]:
legend_titles = ["No pretraining", r"\noindent With cross-lingual \\ pretraining"]
labels = [(True, "score_baseline"), (False, "score_baseline"), (True, "score_crossling"), (False, "score_crossling")]
styles = dict(zip(labels, cycler("color", COLORS) * (cycler("marker", MARKERS) + cycler("ls", LINESTYLES))))

fig, ax = plt.subplots(figsize=(5, 3), gridspec_kw={"left": 0.1, "right": 0.9, "top": 1, "bottom": 0.2})

native, non_native = native_non_native[None]
plot_trajectory(fig, ax, native, non_native, styles)
ax.set_ylabel("ABX accuracy (in \\%)")
ax.set_xlabel("Training hours")
ax.set_title("Development", fontsize="large")

handles, _ = ax.get_legend_handles_labels()
first_legend = fig.legend(
    title=legend_titles[0], handles=[handles[0], handles[1]], loc="lower left", bbox_to_anchor=(0.9, 0.6)
)
first_legend._legend_box.align = "left"
first_legend.get_title().set_color(COLORS[0])
second_legend = fig.legend(
    title=legend_titles[1], handles=[handles[2], handles[3]], loc="lower left", bbox_to_anchor=(0.9, 0.25)
)
second_legend.get_title().set_color(COLORS[1])
second_legend._legend_box.align = "left"

plt.show()

In [ ]:
legend_titles = ["No pretraining", "With cross-lingual pretraining"]
labels = [(True, "score_baseline"), (False, "score_baseline"), (True, "score_crossling"), (False, "score_crossling")]
styles = dict(zip(labels, cycler("color", COLORS) * (cycler("marker", MARKERS) + cycler("ls", LINESTYLES))))

fig, ax = plt.subplots(
    figsize=(6, 4),
    ncols=2,
    nrows=2,
    sharex="all",
    sharey="col",
    gridspec_kw={"wspace": 0.15, "left": 0.15, "right": 1, "top": 0.75, "bottom": 0.15},
)

for i, phone_pair in enumerate([rl, wj]):
    native, non_native = native_non_native[phone_pair]
    for j, col in enumerate(["score_baseline", "score_crossling"]):
        plot_trajectory(None, ax[i, j], native, non_native, {k: v for k, v in styles.items() if k[1] == col})

handles, labels = ax[0, 0].get_legend_handles_labels()
handles = [plt.plot([], marker="", ls="")[0]] + handles
labels = ["Training language: ", "American English", "Japanese"]
legend = fig.legend(
    handles=handles,
    labels=labels,
    loc="lower left",
    bbox_to_anchor=(0.1, 0.85),
    ncols=3,
    frameon=True,
    handlelength=3,
)
legend.legend_handles[1].set_color("black")
legend.legend_handles[2].set_color("black")

for vpack in legend._legend_handle_box.get_children()[:1]:
    for hpack in vpack.get_children():
        hpack.get_children()[0].set_width(0)

ax[0, 0].set_yticks([70, 75, 80, 85, 90, 95])
ax[0, 1].set_yticks([90, 92, 94, 96])
ax[0, 0].set_title(legend_titles[0], color=COLORS[0], fontsize="medium", pad=10)
ax[0, 1].set_title(legend_titles[1], color=COLORS[1], fontsize="medium", pad=10)
ax[0, 1].text(1.1, 0.5, rl.tipa, transform=ax[0, 1].transAxes, fontsize="x-large")
ax[1, 1].text(1.1, 0.5, wj.tipa, transform=ax[1, 1].transAxes, fontsize="x-large")

fig.supylabel("ABX accuracy (in \\%)", x=0.05, y=0.45)
fig.supxlabel("Training hours", x=0.57, y=0)

plt.show()